In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('/content/drive/My Drive/vacs_train.csv',sep=';',index_col=0)

###Data Preprocessing 

In [4]:
drops = ['id','name', 'area.name','city', 'company.id','company', 'company_link', 
         'publication_date', 'salary_currency', 'employment.name', 'schedule.name', 
         'experience.name', 'description', 'type']


def to_int(stri):
    return int(stri)


def change_emplyment(cdf):
    cdf['employment_num'] = (cdf['employment'] == 'full').apply(to_int)
    return cdf.drop('employment', axis = 1)


def change_schecule(cdf):
    type_schedule = list(cdf.groupby('schedule').count().index)
    for typ in type_schedule:
        cdf[typ + '_schedule'] = (cdf['schedule'] == typ).apply(to_int)
    cdf = cdf.drop('schedule', axis = 1)
    return cdf


def delete_dublicates(x):
    return list(dict.fromkeys(x))


def spec_to_int(string):
    spec = string.split()
    res = []
    for num in spec:
        res.append(int(float(num)))
    return delete_dublicates(res)


def add_spec(cdf):
    cdf['spec'] = cdf['specializations'].apply(spec_to_int)
    cdf['spec'] = cdf['spec'].apply(lambda x: x + [0]*6)
    for i in range(6):
        cdf['spec_' + str(i)] = cdf['spec'].apply(lambda x: x[i])
    return cdf


def nor_experience(s):
    if s == 'between1And3':
        return 2
    if s == 'between3And6':
        return 4
    if s == 'moreThan6':
        return 8
    return 0


def change_experience(cdf):
    cdf['experience'] = cdf['experience'].apply(nor_experience)
    return cdf


def get_cur_df(df):
    df1 = df.copy()
    cdf = df1.drop(drops, axis=1)
    cdf = change_emplyment(cdf)
    cdf = change_schecule(cdf)
    cdf = add_spec(cdf)
    cdf = change_experience(cdf)
    return cdf

In [5]:
cdf = get_cur_df(df)
cdf.columns

Index(['name.lemm', 'salary_from', 'experience', 'key_skills',
       'specializations', 'specializations.names', 'description.lemm',
       'employment_num', 'flexible_schedule', 'flyInFlyOut_schedule',
       'fullDay_schedule', 'remote_schedule', 'shift_schedule', 'spec',
       'spec_0', 'spec_1', 'spec_2', 'spec_3', 'spec_4', 'spec_5'],
      dtype='object')

In [6]:
cdf=cdf.drop(['spec', 'key_skills', 'name.lemm', 'specializations','specializations.names','description.lemm'],axis=1)

In [7]:
cdf.head()

,salary_from,experience,employment_num,flexible_schedule,flyInFlyOut_schedule,fullDay_schedule,remote_schedule,shift_schedule,spec_0,spec_1,spec_2,spec_3,spec_4,spec_5
index,,,,,,,,,,,,,,
0,40000.0,2,1,0,0,1,0,0,17,22,0,0,0,0
1,40000.0,2,1,0,0,1,0,0,4,6,2,0,0,0
3,30000.0,2,1,0,0,1,0,0,1,0,0,0,0,0
4,30000.0,2,1,0,0,0,0,1,22,21,17,0,0,0
5,25000.0,4,1,0,0,1,0,0,2,5,0,0,0,0


###Model Setup

In [8]:
a=cdf.columns.get_loc('experience')
b=cdf.columns.get_loc('spec_5')

In [9]:
categorical_columns_index=[]
for i in range (a,b): categorical_columns_index.append(i)
categorical_columns_index

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [10]:
cdf1=cdf.drop(['salary_from'],axis=1)
cdf1

,experience,employment_num,flexible_schedule,flyInFlyOut_schedule,fullDay_schedule,remote_schedule,shift_schedule,spec_0,spec_1,spec_2,spec_3,spec_4,spec_5
index,,,,,,,,,,,,,
0,2,1,0,0,1,0,0,17,22,0,0,0,0
1,2,1,0,0,1,0,0,4,6,2,0,0,0
3,2,1,0,0,1,0,0,1,0,0,0,0,0
4,2,1,0,0,0,0,1,22,21,17,0,0,0
5,4,1,0,0,1,0,0,2,5,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150858,2,1,0,0,1,0,0,17,0,0,0,0,0
150860,4,1,0,0,1,0,0,4,0,0,0,0,0
150861,2,1,0,0,1,0,0,2,0,0,0,0,0


In [11]:
!pip install catboost

In [12]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test =  train_test_split(cdf1,cdf['salary_from'],test_size=0.1, shuffle=True, random_state = 42)

In [14]:
train_pool =  Pool(X_train, y_train, cat_features=categorical_columns_index)
full_train_pool =  Pool(cdf1, cdf['salary_from'], cat_features=categorical_columns_index)

In [15]:
model= CatBoostRegressor(iterations=1000, learning_rate=0.05, bootstrap_type='MVS', depth=16, leaf_estimation_method='Newton', score_function='NewtonL2',
                         od_type='IncToDec', one_hot_max_size=4, l2_leaf_reg=3, random_seed=1)

In [86]:
model.fit(full_train_pool, eval_set=(X_test,y_test), verbose=100)
model.score(X_test,y_test)

0:	learn: 21401.5773737	test: 21464.5682504	best: 21464.5682504 (0)	total: 1.47s	remaining: 24m 27s
100:	learn: 16198.3283763	test: 16224.4991762	best: 16224.3724190 (99)	total: 3m 26s	remaining: 30m 35s
200:	learn: 15891.5661998	test: 16133.2731655	best: 16133.2731655 (200)	total: 7m 43s	remaining: 30m 42s
300:	learn: 15700.0118486	test: 16103.1749709	best: 16103.1749709 (300)	total: 12m 8s	remaining: 28m 11s
400:	learn: 15560.4432387	test: 16089.0642951	best: 16089.0642951 (400)	total: 16m 33s	remaining: 24m 44s
500:	learn: 15440.0232220	test: 16080.4726002	best: 16080.4431742 (499)	total: 20m 57s	remaining: 20m 52s
600:	learn: 15347.9675966	test: 16085.6252285	best: 16078.2295327 (549)	total: 25m 15s	remaining: 16m 46s
700:	learn: 15258.9446343	test: 16084.5174320	best: 16078.2295327 (549)	total: 29m 33s	remaining: 12m 36s
800:	learn: 15183.5176946	test: 16093.8428366	best: 16078.2295327 (549)	total: 33m 47s	remaining: 8m 23s
900:	learn: 15114.0761007	test: 16099.3661672	best: 16078

0.459280892711628

In [87]:
model.save_model('/content/drive/My Drive/cat_model',format="cbm", pool=full_train_pool)

In [16]:
model.load_model('/content/drive/My Drive/cat_model', format='cbm')

###Model Evaluation 

In [17]:
prediction=model.predict(X_test)
prediction

array([35127.88605371, 26614.7341528 , 32285.97168011, ...,
       45061.26288003, 32137.89179896, 38070.75977021])

In [18]:
#округлить до 1к

for i in range(len(prediction)):
  num = prediction[i]
  num = round(num)
  st = str(num)[-5:-2]
  if int(st)>=500:
    prediction[i] = float((int(str(num)[:-5])+1)*1000)
  else: 
    prediction[i] = float((int(str(num)[:-5]))*1000)


In [19]:
prediction

array([35000., 27000., 32000., ..., 45000., 32000., 38000.])

In [20]:
pred = pd.DataFrame({"Original":y_test,"Predicted":prediction}, index=None)

In [26]:
import sklearn.utils
pred = sklearn.utils.shuffle(pred)
pred.head()

,Original,Predicted
index,,
83091,45000.0,45000.0
30447,35000.0,36000.0
57575,27000.0,25000.0
63928,50000.0,51000.0
109981,60000.0,55000.0


###Provided Test Data

In [27]:
tdf=pd.read_csv('/content/drive/My Drive/vacs_test.csv',sep=';',index_col=0)

In [28]:
tdf.shape

(50000, 23)

In [29]:
tdf.head()

,id,name,name.lemm,area.name,city,company.id,company,company_link,publication_date,salary_from,salary_currency,employment,employment.name,schedule,schedule.name,experience,experience.name,key_skills,specializations,specializations.names,description,description.lemm,type
index,,,,,,,,,,,,,,,,,,,,,,,
150869,24030837,Бухгалтер на участок Банковские операции,бухгалтер участок банковский операция,Санкт-Петербург,unknown,1449838,Управляющая компания Технологии Тепличного Роста,https://hh.ru/employer/1449838,2018-01-15T13:54:19+0300,NaN,RUR,full,Полная занятость,fullDay,Полный день,between1And3,От 1 года до 3 лет,NaN,5.45 5.42 5.250,"Банки, инвестиции, лизинг - Валютный контроль ...","В Управляющей компании ""Технологии Тепличного ...",управляющий компания технология тепличный рост...,test
150870,24012268,Помощник бухгалтера,помощник бухгалтер,Санкт-Петербург,Санкт-Петербург,1449838,Управляющая компания Технологии Тепличного Роста,https://hh.ru/employer/1449838,2018-01-09T16:22:00+0300,NaN,RUR,full,Полная занятость,flexible,Гибкий график,noExperience,Нет опыта,NaN,2.179 2.469 15.730,"Бухгалтерия, управленческий учет, финансы пред...","В управляющей компании ""Технологии Тепличного ...",управляющий компания технология тепличный рост...,test
150871,23707419,Заместитель главного бухгалтера,заместитель главный бухгалтер,Санкт-Петербург,Санкт-Петербург,1449838,Управляющая компания Технологии Тепличного Роста,https://hh.ru/employer/1449838,2017-12-13T16:45:16+0300,NaN,RUR,full,Полная занятость,fullDay,Полный день,between3And6,От 3 до 6 лет,NaN,2.43 2.261 2.249,"Бухгалтерия, управленческий учет, финансы пред...","В Управляющей компании ""Технологии Тепличного ...",управляющий компания технология тепличный рост...,test
150872,24306509,Торговый представитель по продажам КГС (компре...,торговый представитель продажа КГС компрессор ...,Санкт-Петербург,unknown,964064,Управляющая компания Техстройконтракт,https://hh.ru/employer/964064,2018-02-27T10:50:04+0300,NaN,RUR,full,Полная занятость,fullDay,Полный день,between3And6,От 3 до 6 лет,Активная жизненная позиция | Коммуникабельност...,7.14 7.222 17.15 17.242 17.306,Автомобильный бизнес - Автозапчасти | Автомоби...,"Обязанности: Продажа компрессоров, генераторо...",обязанность продажа компрессор генератор сварк...,test
150873,24043072,Заместитель директора по сервису,заместитель директор сервис,Санкт-Петербург,unknown,964064,Управляющая компания Техстройконтракт,https://hh.ru/employer/964064,2018-02-13T09:34:20+0300,NaN,RUR,full,Полная занятость,fullDay,Полный день,between3And6,От 3 до 6 лет,NaN,7.267 7.14 7.222 17.15 26.407,Автомобильный бизнес - Сервисное обслуживание ...,Обязанности: Продажа услуг по сервисному обсл...,обязанность продажа услуга сервисный обслужива...,test


In [30]:
tdf = get_cur_df(tdf)
tdf.columns

Index(['name.lemm', 'salary_from', 'experience', 'key_skills',
       'specializations', 'specializations.names', 'description.lemm',
       'employment_num', 'flexible_schedule', 'flyInFlyOut_schedule',
       'fullDay_schedule', 'remote_schedule', 'shift_schedule', 'spec',
       'spec_0', 'spec_1', 'spec_2', 'spec_3', 'spec_4', 'spec_5'],
      dtype='object')

In [31]:
tdf=tdf.drop(['spec', 'key_skills', 'name.lemm', 'specializations','specializations.names','description.lemm'],axis=1)

In [35]:
tdf = tdf.drop(['salary_from'], axis=1)

In [36]:
tdf

,experience,employment_num,flexible_schedule,flyInFlyOut_schedule,fullDay_schedule,remote_schedule,shift_schedule,spec_0,spec_1,spec_2,spec_3,spec_4,spec_5
index,,,,,,,,,,,,,
150869,2,1,0,0,1,0,0,5,0,0,0,0,0
150870,0,1,1,0,0,0,0,2,15,0,0,0,0
150871,4,1,0,0,1,0,0,2,0,0,0,0,0
150872,4,1,0,0,1,0,0,7,17,0,0,0,0
150873,4,1,0,0,1,0,0,7,17,26,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227383,2,1,0,0,1,0,0,5,0,0,0,0,0
227384,2,1,0,0,1,0,0,4,0,0,0,0,0
227385,2,1,0,0,1,0,0,23,0,0,0,0,0


In [37]:
predict_test = model.predict(tdf)

In [40]:
#округлить до 1к

for i in range(len(predict_test)):
  num = predict_test[i]
  num = round(num)
  st = str(num)[-5:-2]
  if int(st)>=500:
    predict_test[i] = float((int(str(num)[:-5])+1)*1000)
  else: 
    predict_test[i] = float((int(str(num)[:-5]))*1000)


In [41]:
pd.DataFrame(predict_test).to_csv("/content/drive/My Drive/result.csv", index=False, header=["Test_Predicted"])

In [42]:
pd.read_csv("/content/drive/My Drive/result.csv")

,Test_Predicted
0,42000.0
1,26000.0
2,49000.0
3,55000.0
4,54000.0
...,...
49995,42000.0
49996,31000.0
49997,39000.0
49998,47000.0
